In [5]:
import re
import os
import gym
import time
import umap
import json
import shlex
import random
import joblib
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from gym import spaces
from tqdm import tqdm
from collections import deque
from scipy.signal import savgol_filter
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("cisco-ai/SecureBERT2.0-biencoder")#載入語意嵌入轉換模型
# =========================
# Path settings
# =========================
pathSAGA = os.path.join("D:\\", "dataset", "SAGA")
pathCasinoLimit = os.path.join("D:\\", "dataset", "CasinoLimit")
pathOutput=os.path.join(pathSAGA,"result")
pathSAGAmalicious=os.path.join(pathSAGA,"SAGA_malicious.json")
print("SAGA exists:", os.path.exists(pathSAGA))

listSAGAFiles=[]
for dirAPTCampaign in os.listdir(pathSAGA):
    pathAPTCampaign=os.path.join(pathSAGA,dirAPTCampaign)
    if os.path.isdir(pathAPTCampaign) and dirAPTCampaign != "result":
        for auditLog in os.listdir(pathAPTCampaign):
            pathAuditLog=os.path.join(pathAPTCampaign,auditLog)
            #print("current processing:",pathAuditLog)
            #print(auditLog)
            if os.path.isfile(pathAuditLog):
                listSAGAFiles.append("path"+"SAGA"+auditLog.split(".")[0])
                globals()["path"+"SAGA"+auditLog.split(".")[0]]=pathAuditLog
print("SAGA所有有的檔案以及其對應路徑")
for i in listSAGAFiles:
    print(i+":"+globals()[i])


SAGA exists: True
SAGA所有有的檔案以及其對應路徑
pathSAGAM1:D:\dataset\SAGA\Composite APT Campaigns Dataset\M1.json
pathSAGAM10:D:\dataset\SAGA\Composite APT Campaigns Dataset\M10.json
pathSAGAM2:D:\dataset\SAGA\Composite APT Campaigns Dataset\M2.json
pathSAGAM3:D:\dataset\SAGA\Composite APT Campaigns Dataset\M3.json
pathSAGAM4:D:\dataset\SAGA\Composite APT Campaigns Dataset\M4.json
pathSAGAM5:D:\dataset\SAGA\Composite APT Campaigns Dataset\M5.json
pathSAGAM6:D:\dataset\SAGA\Composite APT Campaigns Dataset\M6.json
pathSAGAM7:D:\dataset\SAGA\Composite APT Campaigns Dataset\M7.json
pathSAGAM8:D:\dataset\SAGA\Composite APT Campaigns Dataset\M8.json
pathSAGAM9:D:\dataset\SAGA\Composite APT Campaigns Dataset\M9.json
pathSAGAG1:D:\dataset\SAGA\Generated APT Campaigns Dataset\G1.json
pathSAGAG10:D:\dataset\SAGA\Generated APT Campaigns Dataset\G10.json
pathSAGAG11:D:\dataset\SAGA\Generated APT Campaigns Dataset\G11.json
pathSAGAG12:D:\dataset\SAGA\Generated APT Campaigns Dataset\G12.json
pathSAGAG13:D:\dat

In [2]:
def loadSagaJson(path):
    with open(path, "r", encoding="utf-8") as file:
        l = []
        for i in file:
           l.append(json.loads(i)) 
    return l
def sigmoid(x):
    #這邊做了一個避免數字跑出特定範圍的東西
    #這邊承襲原本的版本 如果順利之後可以考慮用原版的去跑跑看
    x =np.clip(x,-500,5.0)
    return 1/(1+np.exp(-x))

def softmax(x, temperature=1.0):
    #同上 先用用看別人的順利再跑跑看不做限制的
    x = np.clip(x, -500, 500)
    x_max = np.max(x)
    exp_x = np.exp((x - x_max) / temperature)
    sum_exp_x = np.sum(exp_x)
    if sum_exp_x == 0:
        return np.ones_like(x) / len(x)
    return exp_x / sum_exp_x
def PCADimemsionalityReduction(embeddings):
    # embeddings: (N, 768)
    X = np.array(embeddings)
    # 1. 標準化（非常重要）
    scaler = StandardScaler()
    X_std = scaler.fit_transform(X)
    # 2. PCA 降到 32 維（RL 很適合）
    pca = PCA(n_components=32, random_state=42)
    X_pca = pca.fit_transform(X_std)
    return X_pca

## RL
這邊設定的類別有兩種，一是「環境」、二是「模型」(或是RL的演算法)
環境取決的載入哪一種紀錄檔案、資料集。如網路封包日誌就會看日誌用甚麼協定、一次傳輸多少bits等訊息，而Audit Log(審計日誌)則會看說使用哪個服務和做了什麼動作。
RL模型則是看要套用哪種演算法和技術
#### 環境設置

### 特徵工程
這部分是關於如何將一條Audit Log轉換成輸入給決策機的數字。
總共有22種特徵 唯有時間戳記(timestamp)可以用縮放的方式轉換成數字，UUID和Pid則不適合這樣處理。其餘皆為字串，使用語意模型轉換成詞嵌入後再進行降維

In [116]:

class AuditLogEnv(gym.Env):
    def __init__(self,logFile="G16.json"):
        # 初始化環境
        ## 基礎設定
        self.logFile=logFile
        #logguardQ有設置size但很明顯我要處理的檔案大很多因此不考慮
        self.stateDim=100 #代表傳入的狀態Dimension有100維
        self.actionDim=4 #代表有四種動作
        ## 接著是載入某份log(這邊預設維G16.json，因為它檔案很小)
        self.logs=self.loadLogs(logFile)#這邊載入會使用 
        if len(self.logs) == 0:#沒有成功載入任何 AuditLog的話會出現錯誤
            raise ValueError(f"No valid log entries found in {logFile}.")
        self.attackPattern=[]#這邊logguardQ給了一個輸入維度在哪種情況下可能是攻擊，但我這邊還不確定怎麼做
        self.noiseLvl=0.05 # 一個加入可能的躁點大概佔幾趴 不確定會不會用到
        ## 以下是代理每次動作後看做了哪些事可能會更新某些值
        self.ipCounts={} # 由於AuditLog也有ip相關可能會用到
        self.logIndex=0 # 看到第幾條
        self.stepCount=0 # 現在走了幾步 
        self.anomalyCount=0
        self.visitedStates=set() # 走訪過那些狀態
        self.processedLogs=set()# 已經處理了哪些東西 作用域: 單次回合 (Single Episode)
        self.processedLogsGlobal=set() #作用域: 整個環境的生命週期 (Entire Lifetime)。
        self.featureRanges={}# 把記錄到的某些數值縮放到某個數字區間 為了提高訓練效率和穩定性。 不確定會不會用到
        # 權重本身可以寫死也可以靠模型自己調整，但我直接拿掉整個權重的設計
        # 主要是我也沒有設備可以弄到自己調整
        #self.featureWeights={}# input的每個東西的權重，不確定要怎麼分配但先放著
        
        ## 常見弱點出現在哪些東西上
        vulnName=[]#常見的有問題的AuditLog可能的特徵 但這邊什麼都沒放 才剛架起來
        attackParamsValues=[]#logGuardQ使用常見的 XSS攻擊會出現的字串特徵 跟上面很像
        attackRegEx=[]# 仿造上面 這邊則用RegEx做做看 一樣還沒放入任何東西
        # 接下來是先載入一次這次回合的(Episode)的AuditLog 並且檢查有多少是非benign
        ## 它的主要功能是在環境載入完所有日誌後，對這些日誌進行初步的分析和統計，以瞭解數據集中異常日誌的「真實分佈」
        ###這邊我還沒改好
        for _, logEntry in self.logs.iterrows():
            if self.isAnomaly(logEntry):
                self.anomalyCount += 1
        print(f"Loaded {len(self.logs)} valid log entries from {logFile}.")
        print(f"Anomaly distribution: {self.anomalyCount}/{len(self.logs)} ({self.anomalyCount/len(self.logs)*100:.1f}% anomalies)")
        self.reset()
    def loadLogs(self,logFile,dataset:str="SAGA"):
        #預設載入SAGA資料集
        #這邊是載入AuditLog的地方，要做成pandas.DataFrame
        listColumns=['relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path']#有甚麼樣的欄位取決於現在讀哪一個資料集
        listLogs=[]
        #讀取檔案
        try:
            with open(logFile,"r",encoding='utf-8')as file:
                for line in file:
                    listLogs.append(json.loads(line))
                    """
                    這邊先沒有用到但先不刪除
                    if lens(part) >=1:
                        listLogs.append(listParts[:11])
                    else:
                        print(f"skipping malformed line:{line.strip()},except 11 fields,got{len(listParts)}")
                        """
            df=pd.json_normalize(listLogs)
            if not listLogs:
                print(f"Warning:no valid log entries found in {logFile}.")
        except FileNotFoundError:
            raise FileNotFoundError(f"File {logFile} not found.")
        except Exception as e:
            raise Exception({f"Error loading file {logFile}: {str(e)}"})
        # 轉換成DataFrame
        df=pd.DataFrame(listLogs,columns=listColumns)
        # 下面則是要把某些部分有固定種類的東西轉換成數值表示
        # 舉例df['status_code'] = pd.to_numeric(df['status_code'], errors='coerce')
        # 特徵擷取分為四種
        #   無大小關係的一組數字(舉例：IP)：計算出現頻率 或 映射到不同數字上
        #   有大小關係的數字(舉例：傳送檔案或字串的大小)：進行歸一化處理(縮放到特定區間)
        #   字串：比對是否與可疑字串有關
        df['relation'] = pd.to_numeric(df['relation'], errors='coerce')
        return df
    def isAnomaly(self,logEntry):
        #對答案
        #這邊logguardQ的實作方法是給了一些很刻意的線索然後比對線索
        #我的實作方式是直接看標籤是不是benign
        #當然我會希望可以做得更好但先這樣
        if logEntry.get('label') == "benign":return False
        else:return True
    def extractFeature(self,logEntry):
        #1 擷取logEntry的各項Value存到各個變數
        #2 特徵標準化以及(加權不做了)
        #3 特徵向量組合(將所有特徵封裝組合成一個numpy陣列)
        listColumns=['relation','timestamp','label','srcNode.UUID','srcNode.Name','srcNode.Image','srcNode.Cmdline','srcNode.Type','srcNode.Pid','dstNode.UUID','dstNode.Key','dstNode.Type','dstNode.Value','dstNode.Name','dstNode.Srcaddress','dstNode.Dstaddress','dstNode.Port','dstNode.Image','dstNode.Cmdline','dstNode.Pid','dstNode','dstNode.Path']
        df=pd.json_normalize(logEntry)
        strLine=""
        for k in df:
            strLine+=f"{k}:{df.at[0,k]}\n"
        embeddings = model.encode(strLine)
        features=np.array(embeddings)
        #normalizedFeatures=np.zeros(features)
        """
        for i,key in enumerate([0,0,0,00]):
            #最後再將所有特徵再次縮放到0,1之間
            minVal,maxVal=self.featureRanges(key)
            normalizedFeatures[i]=np.clip((features[i]-minVal)/(maxVal-minVal+1e-10),0,1)"""
        return features
    def reset(self):
        #1 重製游標/指標
        self.stepCount-0
        #2 清空歷史決策紀錄
        self.processedLogs=set()
        #3 初始化統計用變數
        #4 隨機選一段要跑的Audit log
        #5 回傳初始的State
        self.stepCount=0
        self.processedLogs=set()
        if len(self.logs) > 0:
            unprocessedIndices=[i for i in range(len(self.logs))if i not in self.processedLogsGlobal]
            if unprocessedIndices:
                self.logIndex=random.choice(unprocessedIndices)
            else:
                self.logIndex=np.random.randint(0,len(self.logs))
        else:
            raise IndexError(f"Log Index{self.logIndex} out of range [0,{len(self.logs)}]")
        self.ipCounts={}
        self.visitedStates=set()
        self.state=self.getState()
        return self.state
    def getState(self):
        if 0 <= self.logIndex < len(self.logs):
            logEntry=self.logs.iloc[self.logIndex]
            """
            # ip
            ip=logEntry.get("ip")
            if ip:self.ipCounts[ip]=self.ipCounts.gets(ip,0)+1
            if len(self.ipCounts)>100:
                ipOldest=list(self.ipCounts.keys())[0]
                self.ipCounts[ipOldest]-=1
                if self.ipCounts[ipOldest]==0:
                    del self.ipCounts[ipOldest]"""
            state=self.extractFeature(logEntry)
            self.visitedStates.add(tuple(state))
            return state
        else:
            raise IndexError(f"Log Index {self.logIndex} out of range [0,{len(self.logs)}]")
        return
    def  updateNoise(self):
        self.noiseLvl=0.05
    def step(self,action):
        # 一步
        score=0
        self.stepCount+=1
        if 0<=self.logIndex <len(self.logs):
            logEntry=self.logs.iloc[self.logIndex]
            isAnomaly=self.isAnomaly(logEntry)
            if action == 0 and isAnomaly:
                score = 25.0
                done = False
            elif action == 0 and not isAnomaly:
                score = -15.0
                done = False
            elif action != 0 and isAnomaly:
                score = 2.0
                done = False
            if np.random.rand() <self.noiseLvl:
                score -= 0.5
            self.processedLogs.add(self.logIndex)
            self.logIndex = (self.logIndex + 1) % len(self.logs)
            self.state=self.getState()
            done = self.stepCount>=5 or len(self.processedLogsGlobal)>= self.anomalyCount
            return self.state,score,done,isAnomaly
        else:
            raise IndexError(f"Log Index {self.logIndex} out of range [0,{len(self.logs)}]")
"""
DQN的運作流程
迴圈
    1.環境重製
    2.根據現在狀態選擇一個動作
    3.透過動作與環境運作之後
    4.根據這次互動之後更新權重

"""
class DQN:
    """
    
    """
    def __init__(self,stateDim,actionDim,hiddenDim=128):
        ##同步狀態維度(狀態特徵數量)和動作維度
        self.stateDim=stateDim
        self.actionDim=actionDim
        ##  設定深度學習的網路層數以及參數數量
        self.hiddenDim=hiddenDim
        self.weightS1=np.random.randn(stateDim,hiddenDim)*0.01
        self.weightS2=np.random.randn(hiddenDim,actionDim)*0.01
        self.memory= deque(maxlen=1000)
        ##  各項演算法中的代數設定
        self.gamma =0.99 #折扣因子
        self.epsilon=1.0#  ε-greedy的 ε值 判斷模型該「探索」還是「利用知識」
        self.epsilonMin=0.01# ε的最小值
        self.epsilonDecay=0.998#  ε-greedy的衰減值 模型會逐漸從「探索」轉向利用知識
        self.learnRate=0.002# 學習率
        self.varianceHistory=[]# 紀錄每個回合(episode) 代理所獲得分數的標準差
        self.scoreStates={}#用於儲存分數的平均值和標準差
        self.episode=0
        self.actionHistory=[]
        
    def chooseAction(self,state,visitedStates):
        if np.random.rand() < self.epsilon:
            action = np.random.choice(self.actionDim)
        else:
            qValues=self.forward(state)
            action =np.argmax(qValues)
        curiosityBonus=1.0 if tuple(state) not in visitedStates else 0.0
        ## 如果採取一個新動作的時候會觸發一個好奇心獎勵值
        self.actionHistory.append(action)
        return action,curiosityBonus
    def forward(self,state):
        #用矩陣乘法把狀態乘上第一層權重再用sigmoid(隱藏層的活化函數)。再來矩陣乘法乘上第二層權重，向量中的每一個值就是神經網路預測的在當狀態下採取特定動作的Q值
        hidden = sigmoid(np.dot(state,self.weightS1))
        return np.dot(hidden,self.weightS2)
    def updateWeights(self,state,action,reward,prevState,curiosityBonus):
        nextQValue=self.forward(state)
        target=reward+self.gamma*np.max(nextQValue)+curiosityBonus
        
        prevHidden=sigmoid(np.dot(prevState,self.weightS1))
        prevQ= np.dot(prevHidden,self.weightS2)[action]
        
        delta = target-prevQ
        
        deltaOutput = np.zeros(self.actionDim)
        deltaOutput[action] =delta
        
        gradW2 =np.outer(prevHidden,deltaOutput)
        deltaHidden=(
            np.dot(self.weightS2,deltaOutput)
            *prevHidden
            *(1-prevHidden)
        )
        gradW1=np.outer(prevState,deltaHidden)
        """
        currentQ=self.forward(prevState)[action]
        delta=target-currentQ
        hidden=sigmoid(np.dot(prevState,self.weightS1))
        deltaOutput=np.zeros(self.actionDim)
        deltaOutput[action]=delta
        grade2=np.outer(prevState,deltaOutput)
        deltaHidden=np.dot(self.weightS2,deltaOutput)*hidden*(1-hidden)
        grade1=np.outer(prevState,deltaHidden)
        """
        self.weightS1+=self.learnRate*gradW1
        self.weightS2+=self.learnRate*gradW2
        self.memory.append((prevState,action,reward,state))
        self.epsilon=max(self.epsilonMin,self.epsilon*self.epsilonDecay)
        self.scoreStates['mean']=self.scoreStates.get('mean',0)*0.99+reward*0.01
        self.scoreStates['std']=np.sqrt(self.scoreStates.get('std',0)**2*0.99+(reward-self.scoreStates['mean'])**2*0.01)
        self.varianceHistory.append(self.scoreStates['std'])
    def reportDetection(self,isAnomalyStep,action):
        #這個不知道在幹嘛的
        pass

"""

"""
class PPO:
    def __init__(self):
        return
    def chooseAction():
        return
    def forward():
        return
    def updateWeights():
        return
    def reportDetection():
        pass
def evaluateStatisticalSignificance():
    #針對不同的RL模型進行更嚴謹的性能和統計學比較
    #實作與功能
    # 先為每個模型的得分計算平均數 中位數 標準差 常態分佈檢定
    # 再來選擇並且統計檢定 比較每兩個東西模型的表現
    # 最後印出剩下的結果
    return
def runSimulation(model,env,modelName,episodes,maxSteps):
    #1 初始化變數
    scores=[]
    f1Scores=[]
    stepsToDetection=[]
    detections=0
    truePositives=0
    falsePositives=0
    trueNegatives=0
    falseNegatives=0
    totalAnomalies=0
    totalStart=time.time()
    cumulativeTp=[]
    #2 迴圈
    for episode in range(episodes):
        #2.1 提前停止條件
        if len(env.processedLogsGlobal)>= env.anomalyCount:
            #如果已經跑完全部的異常的話即停止
            print(f"{modelName} stopped early at episode {episode}: All {env.anomalyCount} anomalies processed.")
            break
        #每個episode開始 重置環境 清空episode統計
        state = env.reset()
        totalScore = 0
        steps=  0
        done = False
        episodeTruePositives=0
        episodeAnomalies=0
        actionCounts=np.zeros(4)
        #每一百步會更新一次躁點避免死記硬背
        if episode % 100 ==0:
            env.updateNoise()
        #確認model有 episode
        if hasattr(model,"episode"):
            model.episode=episode
        #   model與環境互動
        while not done and steps < maxSteps:
            prevState=state#記住前一個state
            #選擇動作(choosing action)
            action,curiosityBonus=model.chooseAction(state,env.visitedStates)
            actionCounts[action]+=1
            #取出當前的log並且判斷是否是anomaly
            logEntry=env.logs.iloc[env.logIndex]
            isAnomaly=env.isAnomaly(logEntry)
            if isAnomaly and env.logIndex not in env.processedLogsGlobal:
                episodeAnomalies+=1
                totalAnomalies+=1
                env.processedLogsGlobal.add(env.logIndex)
            nextState,score,done,isAnomalyStep=env.step(action)
            #執行動作
            if action == 0 and isAnomaly:
                episodeTruePositives+=1
                truePositives+=1
            elif action == 0 and not isAnomaly:
                falsePositives+=1
            elif action != 0 and isAnomaly:
                falseNegatives+=1
            elif action != 0 and not isAnomaly:
                trueNegatives+=1
            #更新權重
            model.updateWeights(nextState,action,score,prevState,curiosityBonus)
            model.reportDetection(isAnomalyStep,action)
            #更新狀態
            state = nextState
            totalScore += score
            step += 1
        #分類成功就加一
        if episodeTruePositives > 0:
            detection+=1
        #計算回合分數
        scores.append(totalScore)
        precision = episodeTruePositives / (episodeTruePositives+falsePositives)if (episodeTruePositives+falsePositives)>0 else 0
        recall = episodeTruePositives/(episodeTruePositives+falseNegatives)if (episodeTruePositives+falseNegatives)>0 else 0
        f1 = 2 * (precision*recall)/(precision+recall) if (precision+recall)>0 else 0
        #更新該回合累積
        f1Scores.append(f1)
        stepsToDetection.append(steps)
        cumulativeTp.append(truePositives)
        if episode % 500 == 0 or episode == episodes -1 or len(env.processedLogsGlobal)>-env.anomalyCount:
            precision = truePositives / ( truePositives+falsePositives) if (truePositives+falsePositives) > 0 else 0
            recall = truePositives / (truePositives+ falseNegatives)if (truePositives+trueNegatives) >0 else 0
            f1Score = 2 * (precision*recall)/(precision+ recall)if (precision+recall) > 0 else 0
            print(f"""
                  {modelName} Episode {episode}: Detection {detections}/{episode+1}({detection/(episode+1)*100:.1f}")
                  True Positives: {truePositives}, False Positives: {falsePositives}
                  True Negative: {trueNegatives}, False Negatives: {falseNegatives}
                  Total Anomalies Encounter:{totalAnomalies} (Episode Anomalies: {episodeAnomalies})
                  Global Processed Logs: {len(env.processedLogsGlobal)}
                  Precision: {precision:.4f},Recall: {recall:.4f},F1-score: {f1Score:4f}
                  Action Distribution: {actionCounts /actionCounts.sum()}
                  """)
        #計算運算時間
        totalTime= time.time()- totalStart
    return {"scores":scores,"f1Scores":f1Scores,"stepsToDetection":stepsToDetection,"detections":detections,"truePositives":truePositives,"falsePositive":falsePositives,"falseNegatives":falseNegatives,"totalAnomalies":totalAnomalies,"cumulativesTp":cumulativeTp,"actionHistory":model.actionHistory}
def displayResult(model="",dictResult:dict={}):
    precision=dictResult['tp']/(dictResult['tp']+dictResult['fp'])
    recall=dictResult['tp']/(dictResult['tp']+dictResult['fn'])
    f1Score=2*(precision*recall)/(precision+recall)
    successfulSteps=[s for s in dictResult['steps'][-100:]if s<5]
    subTitle=f"""
{model} Final Result
=====================
precision:{precision}
recall:{recall}
detection:{dictResult['detection']/len(dictResult['scores'])}({dictResult['detection']}/
{len(dictResult['scores'])*100:.1f}%)
true positive:{dictResult['tp']}
false positive:{dictResult['fp']}
true negative:{dictResult['tn']}
false negative:{dictResult['fn']}
total anomalies encountered:{dictResult['anomalies']}
({dictResult['anomalies']/dictResult['tp']+dictResult['tn']+dictResult['fp']+dictResult['fn']*100:.1f}% of steps)
Mean Reward (all episodes): {np.mean(dictResult['scores']):.4f} ± {np.std(dictResult['scores']):.4f}
Mean F1-Score (all episodes): {np.mean(f1Score):.4f} ± {np.std(f1Score):.4f}
Mean Steps per episode (all episodes): {np.mean(dictResult['steps']):.1f} ± {np.std(dictResult['steps']):.1f}
Metrics for last 100 Episodes:
Mean Reward (last 100): {np.mean(dictResult['scores'][-100:]):.4f} ± {np.std(dictResult['scores'][-100:]):.4f}
Mean F1-Scores (last 100): {np.mean(f1Score[-100:]):.4f} ± {np.std(f1Score[-100:]):.4f}
Steps to Detection (last 100, successful episodes): {np.mean(successfulSteps):.1f} ± {np.std(successfulSteps):.1f}
"""if successfulSteps else "No Successful episodes"
    print(subTitle)
    return
def evaluateStatisticalSignificance():
    #針對不同的RL模型進行更嚴謹的性能和統計學比較
    #實作與功能
    # 先為每個模型的得分計算平均數 中位數 標準差 常態分佈檢定
    # 再來選擇並且統計檢定 比較每兩個東西模型的表現
    # 最後印出剩下的結果
    return
def runSimulation(model,env,modelName,episodes,maxSteps):
    #1 初始化變數
    scores=[]
    f1Scores=[]
    stepsToDetection=[]
    detections=0
    truePositives=0
    falsePositives=0
    trueNegatives=0
    falseNegatives=0
    totalAnomalies=0
    totalStart=time.time()
    cumulativeTp=[]
    #2 迴圈
    for episode in range(episodes):
        #2.1 提前停止條件
        if len(env.processedLogsGlobal)>= env.anomalyCount:
            #如果已經跑完全部的異常的話即停止
            print(f"{modelName} stopped early at episode {episode}: All {env.anomalyCount} anomalies processed.")
            break
        #每個episode開始 重置環境 清空episode統計
        state = env.reset()
        totalScore = 0
        steps=  0
        done = False
        episodeTruePositives=0
        episodeAnomalies=0
        actionCounts=np.zeros(4)
        #每一百步會更新一次躁點避免死記硬背
        if episode % 100 ==0:
            env.updateNoise()
        #確認model有 episode
        if hasattr(model,"episode"):
            model.episode=episode
        #   model與環境互動
        while not done and steps < maxSteps:
            prevState=state#記住前一個state
            #選擇動作(choosing action)
            action,curiosityBonus=model.chooseAction(state,env.visitedStates)
            actionCounts[action]+=1
            #取出當前的log並且判斷是否是anomaly
            logEntry=env.logs.iloc[env.logIndex]
            isAnomaly=env.isAnomaly(logEntry)
            if isAnomaly and env.logIndex not in env.processedLogsGlobal:
                episodeAnomalies+=1
                totalAnomalies+=1
                env.processedLogsGlobal.add(env.logIndex)
            nextState,score,done,isAnomalyStep=env.step(action)
            #執行動作
            if action == 0 and isAnomaly:
                episodeTruePositives+=1
                truePositives+=1
            elif action == 0 and not isAnomaly:
                falsePositives+=1
            elif action != 0 and isAnomaly:
                falseNegatives+=1
            elif action != 0 and not isAnomaly:
                trueNegatives+=1
            #更新權重
            model.updateWeights(nextState,action,score,prevState,curiosityBonus)
            model.reportDetection(isAnomalyStep,action)
            #更新狀態
            state = nextState
            totalScore += score
            steps += 1
        #分類成功就加一
        if episodeTruePositives > 0:
            detection+=1
        #計算回合分數
        scores.append(totalScore)
        precision = episodeTruePositives / (episodeTruePositives+falsePositives)if (episodeTruePositives+falsePositives)>0 else 0
        recall = episodeTruePositives/(episodeTruePositives+falseNegatives)if (episodeTruePositives+falseNegatives)>0 else 0
        f1 = 2 * (precision*recall)/(precision+recall) if (precision+recall)>0 else 0
        #更新該回合累積
        f1Scores.append(f1)
        stepsToDetection.append(steps)
        cumulativeTp.append(truePositives)
        if episode % 500 == 0 or episode == episodes -1 or len(env.processedLogsGlobal)>-env.anomalyCount:
            precision = truePositives / ( truePositives+falsePositives) if (truePositives+falsePositives) > 0 else 0
            recall = truePositives / (truePositives+ falseNegatives)if (truePositives+falseNegatives) >0 else 0
            f1Score = 2 * (precision*recall)/(precision+ recall)if (precision+recall) > 0 else 0
            print(f"""
                  {modelName} Episode {episode}: Detection {detections}/{episode+1}({detections/(episode+1)*100:.1f}")
                  True Positives: {truePositives}, False Positives: {falsePositives}
                  True Negative: {trueNegatives}, False Negatives: {falseNegatives}
                  Total Anomalies Encounter:{totalAnomalies} (Episode Anomalies: {episodeAnomalies})
                  Global Processed Logs: {len(env.processedLogsGlobal)}
                  Precision: {precision:.4f},Recall: {recall:.4f},F1-score: {f1Score:4f}
                  Action Distribution: {actionCounts /actionCounts.sum()}
                  """)
        #計算運算時間
        totalTime= time.time()- totalStart
    return {"scores":scores,"f1Scores":f1Scores,"stepsToDetection":stepsToDetection,"detections":detections,"truePositives":truePositives,"falsePositive":falsePositives,"falseNegatives":falseNegatives,"totalAnomalies":totalAnomalies,"cumulativesTp":cumulativeTp,"actionHistory":model.actionHistory}
def displayResult(model="",dictResult:dict={}):
    precision=dictResult['tp']/(dictResult['tp']+dictResult['fp'])
    recall=dictResult['tp']/(dictResult['tp']+dictResult['fn'])
    f1Score=2*(precision*recall)/(precision+recall)
    successfulSteps=[s for s in dictResult['steps'][-100:]if s<5]
    subTitle=f"""
{model} Final Result
=====================
precision:{precision}
recall:{recall}
detection:{dictResult['detection']/len(dictResult['scores'])}({dictResult['detection']}/
{len(dictResult['scores'])*100:.1f}%)
true positive:{dictResult['tp']}
false positive:{dictResult['fp']}
true negative:{dictResult['tn']}
false negative:{dictResult['fn']}
total anomalies encountered:{dictResult['anomalies']}
({dictResult['anomalies']/dictResult['tp']+dictResult['tn']+dictResult['fp']+dictResult['fn']*100:.1f}% of steps)
Mean Reward (all episodes): {np.mean(dictResult['scores']):.4f} ± {np.std(dictResult['scores']):.4f}
Mean F1-Score (all episodes): {np.mean(f1Score):.4f} ± {np.std(f1Score):.4f}
Mean Steps per episode (all episodes): {np.mean(dictResult['steps']):.1f} ± {np.std(dictResult['steps']):.1f}
Metrics for last 100 Episodes:
Mean Reward (last 100): {np.mean(dictResult['scores'][-100:]):.4f} ± {np.std(dictResult['scores'][-100:]):.4f}
Mean F1-Scores (last 100): {np.mean(f1Score[-100:]):.4f} ± {np.std(f1Score[-100:]):.4f}
Steps to Detection (last 100, successful episodes): {np.mean(successfulSteps):.1f} ± {np.std(successfulSteps):.1f}
"""if successfulSteps else "No Successful episodes"
    print(subTitle)
    return
    

# 主程式

In [15]:
if __name__ == "__main__" :
    
    """#先印出某個Audit Log檔案的前十行
    fileNameM1=listSAGAFiles[0].split("th")[1]
    print(f"Preview the first 10 lines in file{fileNameM1}")
    listLogsM1=loadSagaJson(globals()[listSAGAFiles[0]])
    for i in range(10):print(listLogsM1[i])
    """
    np.random.seed(42)
    #環境設定初始化
    env=AuditLogEnv(logFile=globals()[listSAGAFiles[0]])
    #RL模型初始化
    dqnModel=DQN(stateDim=768,actionDim=4)
    #回合(Episode)設定初始化
    
    #開始進行模擬
    
    #評估結果
    
    #印出結果

Loaded 694858 valid log entries from D:\dataset\SAGA\Composite APT Campaigns Dataset\M1.json.
Anomaly distribution: 694858/694858 (100.0% anomalies)


In [117]:
#環境設定初始化
env=AuditLogEnv(logFile=globals()[listSAGAFiles[0]])

Loaded 694858 valid log entries from D:\dataset\SAGA\Composite APT Campaigns Dataset\M1.json.
Anomaly distribution: 233/694858 (0.0% anomalies)


In [118]:
#RL模型初始化
dqnModel=DQN(stateDim=768,actionDim=4)

In [120]:
#回合(Episode)設定初始化
episodes=2000    
#開始進行模擬
resultDQN=runSimulation(model=dqnModel,env=env,modelName="DQN",episodes=episodes,maxSteps=2000)


                  DQN Episode 0: Detection 0/1(0.0")
                  True Positives: 0, False Positives: 0
                  True Negative: 5, False Negatives: 0
                  Total Anomalies Encounter:0 (Episode Anomalies: 0)
                  Global Processed Logs: 1
                  Precision: 0.0000,Recall: 0.0000,F1-score: 0.000000
                  Action Distribution: [0. 0. 1. 0.]
                  

                  DQN Episode 1: Detection 0/2(0.0")
                  True Positives: 0, False Positives: 0
                  True Negative: 10, False Negatives: 0
                  Total Anomalies Encounter:0 (Episode Anomalies: 0)
                  Global Processed Logs: 1
                  Precision: 0.0000,Recall: 0.0000,F1-score: 0.000000
                  Action Distribution: [0. 0. 1. 0.]
                  

                  DQN Episode 2: Detection 0/3(0.0")
                  True Positives: 0, False Positives: 0
                  True Negative: 15, False Negative

KeyboardInterrupt: 

In [ ]:

    #評估結果
    
    #印出結果